In [ ]:
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-1.0-py3-none-any.whl

In [ ]:
import spacy
nlp = spacy.load('tr_core_news_trf')

In [12]:
# bu entity lerin Aspect Based modelden gelmesi beklenmektedir.
entities1 = ["Vodafone", "Turkcell", "Türk Telekom"]

def get_split_sentences_with_conjunction(text):

    doc = nlp(text) # metnin tokenize haline getirilmesi
    split_sentences = [] # çıkarılan cümlelerin tutulacağı dizinin başlatılması
    add_old_entity = False # ilk durumda eski entity eklenmesi bool değeri False verilir.
    for sent in doc.sents: # metnin her bir cümlesi için:
        conjunctions = [token for token in sent if token.pos_ in ['CCONJ', 'SCONJ'] or token.text == "\0"] # bağlaçların metin içinden tanınması
        entities = [token for token in sent if token.text in entities1] # entity lerin metin içinden tanınması
        k = 0 # ilk bağlaç için bir sayaç oluşturulması
        if conjunctions: # metinde bağlaç var ise:
          for conjunction in conjunctions: # her bir bağlaç için:
            if k==0: # ilk bağlaç için:
              if sent[conjunction.i-1] in entities and sent[conjunction.i+1] in entities: # eğer bağlaçtan bir önceki ve sonraki kelime bir entity ise:
                old_entity = sent[conjunction.i-1] # önceki entity bir değişkende tutulur
                add_old_entity = True # eski entity sonradan ekleneceği için bool değişkeni True yapılır
              else: # eğer bağlaçtan bir önceki ve sonraki kelime bir entity değil ise:
                text = sent[:conjunction.i].text.strip() # bağlaçtan önceki cümle metinden alınır
                split_sentences.append(text) # alınan metin diziye eklenir
              k += 1 # ilk bağlaçtan çıkıldığı için sayaç arttırılır
              old_conjunction = conjunction # eski bağlaç bir değişkende tutulur
            else: # ilk bağlaçtan sonraki bağlaçlar ise:
              if sent[conjunction.i-1] in entities and sent[conjunction.i+1] in entities: # eğer bağlaçtan bir önceki ve sonraki kelime bir entity ise:
                old_entity = sent[conjunction.i-1] # önceki entity bir değişkende tutulur
                add_old_entity = True # eski entity sonradan ekleneceği için bool değişkeni True yapılır
              else: # eğer bağlaçtan bir önceki ve sonraki kelime bir entity değil ise:
                text = sent[old_conjunction.i+1:conjunction.i].text.strip() # eski bağlaçtan şu anki bağlaça kadar olan cümle metinden alınır
                split_sentences.append(text) # alınan metin diziye eklenir
                if add_old_entity == True: # eski entity nin eklenmesi gerekiyor ise:
                    text = sent[old_entity.i].text.strip() # eski entity cümleden alınır
                    text2 = sent[old_conjunction.i+2:conjunction.i].text.strip() # eski bağlaçtan ve entityden sonra şu anki bağlaça kadar olan cümle metinden alınır
                    split_sentences.append(text + " " + text2) # eski entity ile alınan cümle arasına bir boşluk eklenerek diziye eklenir
                    add_old_entity = False # eski entity alındığı için bool False değerine getirilir
              old_conjunction = conjunction # eski bağlaç bir değişkende tutulur
              k += 1
          split_sentences.append(sent[conjunction.i + 1:].text.strip())

    while("" in split_sentences):
      split_sentences.remove("")

    return split_sentences

# Ö R N E K L E R
text = "Türk Telekom kötü ama Vodafone ve Turkcell çok iyi."
split_sentences = get_split_sentences_with_conjunction(text)
print(split_sentences)

text = "Vodafone ve Turkcell çok iyi ama Türk Telekom kötü."
split_sentences = get_split_sentences_with_conjunction(text)
print(split_sentences)

text = "Türk Telekom kötü ama Vodafone güzel ve Turkcell çok iyi."
split_sentences = get_split_sentences_with_conjunction(text)
print(split_sentences)

text = "Turkcell Vodafone güzelken çok iyi ama Türk Telekom tercih ederek hata yaptım"
split_sentences = get_split_sentences_with_conjunction(text)
print(split_sentences)

text = "Turkcell ve Vodafone güzelken çok iyi ama Türk Telekom tercih ederek hata yaptım"
split_sentences = get_split_sentences_with_conjunction(text)
print(split_sentences)

['Türk Telekom kötü', 'Turkcell çok iyi.']
['Turkcell çok iyi', 'Vodafone çok iyi', 'Türk Telekom kötü.']
['Türk Telekom kötü', 'Vodafone güzel', 'Turkcell çok iyi.']
['Turkcell Vodafone güzelken çok iyi', 'Türk Telekom tercih ederek hata yaptım']
['Vodafone güzelken çok iyi', 'Turkcell güzelken çok iyi', 'Türk Telekom tercih ederek hata yaptım']
